In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import re

# Load the dataframe
df = pd.read_csv("data.csv")

tool_col = "Which of the following AI tools have you used for academic tasks in the past 30 days? (Select all that apply)"

# Split and count tools
tool_list = df[tool_col].dropna().str.split(";").sum()
tool_counts = Counter([tool.strip() for tool in tool_list if tool.strip()])

tool_df = pd.DataFrame(tool_counts.items(), columns=["Tool", "Count"]).sort_values(
    "Count", ascending=False
)

tool_df["Tool"] = tool_df["Tool"].str.replace(r"\s\(e\.g\..*?\)", "", regex=True)

fig, ax = plt.subplots(figsize=(14, 9))  # Create figure and axes together
tool_df.plot(kind="barh", x="Tool", y="Count", legend=False, ax=ax)

ax.set_title("Popularity of AI Tools", fontsize=14)
ax.set_xlabel("Number of Users", fontsize=12)
ax.set_ylabel("AI Tool", fontsize=12)
ax.tick_params(axis="x", labelsize=12)
ax.tick_params(axis="y", labelsize=14)
ax.invert_yaxis()

max_count = int(tool_df["Count"].max()) if not tool_df["Count"].empty else 0
if max_count > 10:
    ax.set_xticks(range(0, max_count + 2, 2))
else:
    ax.set_xticks(range(0, max_count + 2, 1))

plt.tight_layout()
plt.savefig("popularity_bar.png", dpi=300, bbox_inches="tight")
plt.clf()

print("Tool counts:\n", tool_df)

# Process Academic Tasks

task_col = "For which of the following academic tasks do you use AI tools? (Select all that apply)"

task_list = df[task_col].dropna().str.split(";").sum()
task_counts = Counter([task.strip() for task in task_list if task.strip()])

task_df = pd.DataFrame(task_counts.items(), columns=["Task", "Count"]).sort_values(
    "Count", ascending=False
)

task_df_sorted = task_df.sort_values("Count", ascending=True)
total_task_responses = task_df_sorted["Count"].sum()
task_df_sorted["percentage"] = (task_df_sorted["Count"] / total_task_responses) * 100

threshold = 2.0
small_tasks = task_df_sorted[task_df_sorted["percentage"] < threshold]
main_tasks = task_df_sorted[task_df_sorted["percentage"] >= threshold]

if not small_tasks.empty:
    other_count = small_tasks["Count"].sum()
    other_df = pd.DataFrame([{"Task": "Other", "Count": other_count}])
    main_tasks = pd.concat([main_tasks, other_df])

plt.figure(figsize=(12, 10))
main_tasks.set_index("Task").plot.pie(
    y="Count",
    autopct="%1.1f%%",
    startangle=90,
    legend=False,
    fontsize=7,
    pctdistance=0.85,
)
plt.title("Distribution of AI Use for Academic Tasks", fontsize=12)
plt.ylabel("")
centre_circle = plt.Circle((0, 0), 0.70, fc="white")
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.tight_layout()
plt.savefig("distribution_donut.png", dpi=300, bbox_inches="tight")
plt.clf()

print("\nTask counts:\n", task_df)

# Efficiency Chart

col_name = "To what extent do you agree with the following statement: 'AI tools help me complete academic tasks more efficiently.'"

if col_name in df.columns:
    print(f"\nValue counts for '{col_name}':")
    print(df[col_name].value_counts())
    print("\n")

    counts = df[col_name].value_counts().sort_index()

    efficiency_df = pd.DataFrame(
        {"Agreement Level": counts.index, "Count": counts.values}
    )

    fig, ax = plt.subplots(figsize=(10, 6))
    efficiency_df.plot(
        kind="bar", x="Agreement Level", y="Count", legend=False, ax=ax, color="skyblue"
    )

    ax.set_title("Student Perceived Efficiency of AI Tools", fontsize=14)
    ax.set_xlabel(
        "Agreement Level (e.g., 1=Strongly Disagree, 5=Strongly Agree)", fontsize=12
    )
    ax.set_ylabel("Number of Responses", fontsize=12)
    ax.tick_params(axis="x", labelsize=12, rotation=0)
    ax.tick_params(axis="y", labelsize=12)
    ax.set_ylim(0, 20)
    ax.set_yticks(range(0, 21, 2))
    ax.grid(axis="y", linestyle="--", alpha=0.7)

    plt.tight_layout()
    plt.savefig("efficiency_chart.png", dpi=300, bbox_inches="tight")
    plt.clf()

else:
    print(f"Error: Column '{col_name}' not found in the CSV.")
    print("Please check the column name for typos or variations.")


Tool counts:
                            Tool  Count
2        Generative AI for text     29
4    Grammar/Writing assistance     25
1     Note-taking/summarization     24
0    Code generation/completion     17
6   Data analysis/visualization     14
7  Generative AI for images/art     13
5                   Translation      8
3                         Other      3

Task counts:
                        Task  Count
4         Summarizing texts     29
3  Outlining essays/reports     27
7       Research assistance     26
0       Brainstorming ideas     24
1      Proofreading/Grammar     20
5             Data analysis     17
2  Generating code snippets     14
6     Translating languages     10
8                     Other      4

Value counts for 'To what extent do you agree with the following statement: 'AI tools help me complete academic tasks more efficiently.'':
To what extent do you agree with the following statement: 'AI tools help me complete academic tasks more efficiently.'
4    17
3  

<Figure size 1400x900 with 0 Axes>

<Figure size 1200x1000 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 1000x600 with 0 Axes>